In [1]:
pip install transformers


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 6.8 MB 2.6 MB/s            
     |████████████████████████████████| 17.3 MB 61.2 MB/s            
     |████████████████████████████████| 7.6 MB 39.5 MB/s            
     |████████████████████████████████| 199 kB 51.4 MB/s            
     |████████████████████████████████| 769 kB 57.6 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install torch


Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 12.2 MB 5.5 MB/s            


In [4]:
!pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 9.6 MB 2.2 MB/s            
     |████████████████████████████████| 34.5 MB 12.4 MB/s            
     |████████████████████████████████| 297 kB 15.2 MB/s            


In [1]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [2]:
df = pd.read_csv("main_data.csv")

In [3]:
df

,Unnamed: 0,content,label
0,0,Researchers use deep learning to simulate chlo...,Positive
1,1,Why is our @Conservatives government so evil?\...,Negative
2,2,"Texas Oilfield Waste Company Contributed $53,7...",Negative
3,3,Epic California snowpack is now the deepest it...,Negative
4,4,If #climatechange is real and not a hoax why d...,Negative
...,...,...,...
4405,353,['𝐃𝐚𝐭𝐚 𝐃𝐫𝐢𝐯𝐞𝐧 𝐃𝐄𝐈 with @ team73bit (73bit. com...,Neutral
4406,354,['𝐃𝐚𝐭𝐚 𝐃𝐫𝐢𝐯𝐞𝐧 𝐃𝐄𝐈 with @ team73bit (73bit. com...,Neutral
4407,355,['Motivation direction on how to discover # su...,Positive
4408,356,['𝐃𝐚𝐭𝐚 𝐃𝐫𝐢𝐯𝐞𝐧 𝐃𝐄𝐈 with @ team73bit (73bit. com...,Neutral


In [4]:
df['label']

0       Positive
1       Negative
2       Negative
3       Negative
4       Negative
          ...   
4405     Neutral
4406     Neutral
4407    Positive
4408     Neutral
4409    Positive
Name: label, Length: 4410, dtype: object

In [5]:
# Clean up label column
df["label"] = df["label"].str.strip().str.lower()



In [6]:
df["label"] 

0       positive
1       negative
2       negative
3       negative
4       negative
          ...   
4405     neutral
4406     neutral
4407    positive
4408     neutral
4409    positive
Name: label, Length: 4410, dtype: object

In [7]:
# Map labels to numerical values
label_map = {"positive": 1, "negative": 0, "neutral": 2}
df["label"] = df["label"].map(label_map)


In [8]:
df["label"] 

0       1
1       0
2       0
3       0
4       0
       ..
4405    2
4406    2
4407    1
4408    2
4409    1
Name: label, Length: 4410, dtype: int64

In [9]:



# Split dataset into train and test sets
train_text, test_text, train_labels, test_labels = train_test_split(df["content"], df["label"], test_size=0.2)

# Tokenize the texts
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_encodings = tokenizer(train_text.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_text.tolist(), truncation=True, padding=True)

# Convert labels to tensors
train_labels = torch.tensor(train_labels.tolist())
test_labels = torch.tensor(test_labels.tolist())

# Create datasets
train_dataset = TensorDataset(torch.tensor(train_encodings["input_ids"]),
                              torch.tensor(train_encodings["attention_mask"]),
                              train_labels)
test_dataset = TensorDataset(torch.tensor(test_encodings["input_ids"]),
                             torch.tensor(test_encodings["attention_mask"]),
                             test_labels)

In [10]:
df['label']

0       1
1       0
2       0
3       0
4       0
       ..
4405    2
4406    2
4407    1
4408    2
4409    1
Name: label, Length: 4410, dtype: int64

In [11]:
# Initialize BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

# Set training parameters
batch_size = 64
epochs = 4
learning_rate = 2e-5
adam_epsilon = 1e-8

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, eps=adam_epsilon)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [16]:
# Train the model
for epoch in range(epochs):
    print(f"Epoch {epoch+1}")
    model.train()

    train_loss = 0
    train_preds = []
    train_true = []

    for batch in train_dataloader:
        inputs = {"input_ids": batch[0],
                  "attention_mask": batch[1],
                  "labels": batch[2].reshape(-1, 1)}  # Reshape the target tensor

        optimizer.zero_grad()

        outputs = model(**inputs)  # Pass inputs as keyword arguments

        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_preds += torch.argmax(logits, axis=1).tolist()
        train_true += batch[2].tolist()

    train_acc = accuracy_score(train_true, train_preds)
    train_p, train_r, train_f1, _ = precision_recall_fscore_support(train_true, train_preds, average="macro")

    print(f"Train loss: {train_loss/len(train_dataloader)}")
    print(f"Train accuracy: {train_acc}")


Epoch 1
Train loss: 1.0154521018266678
Train accuracy: 0.4336734693877551
Epoch 2
Train loss: 0.9538193675024169
Train accuracy: 0.5232426303854876
Epoch 3
Train loss: 0.8298185116478375
Train accuracy: 0.626984126984127
Epoch 4
Train loss: 0.6285783020513398
Train accuracy: 0.7531179138321995


In [20]:
 # Evaluate the model on the test set
model.eval()

test_loss = 0
test_preds = []
test_true = []

for batch in test_dataloader:
        inputs = {"input_ids": batch[0],
                  "attention_mask": batch[1],
                  "labels": batch[2].reshape(-1, 1)}

        with torch.no_grad():
            outputs = model(**inputs)

            loss = outputs.loss
            logits = outputs.logits

            test_loss += loss.item()
            test_preds += torch.argmax(logits, axis=1).tolist()
            test_true += batch[2].tolist()
            
            test_acc = accuracy_score(test_true, test_preds)
            test_p, test_r, test_f1, _ = precision_recall_fscore_support(test_true, test_preds, average="macro")

print(f"Test loss: {test_loss/len(test_dataloader)}")
print(f"Test accuracy: {test_acc}")
print(f"Test precision: {test_p}")
print(f"Test recall: {test_r}")
print(f"Test F1 score: {test_f1}")

Test loss: 0.8477515578269958
Test accuracy: 0.6337868480725624
Test precision: 0.635219045812161
Test recall: 0.6032340911071343
Test F1 score: 0.6144473699800513
